In [1]:
from torchvision import transforms
import time
import utils
import random
import numpy
import torch.nn.functional as F
import torch
import torch.nn as nn
import torchvision
trainfile = '/home/labs/testing/class65'
import os
import sys
sys.path.append(os.path.dirname(os.path.expanduser(trainfile)))
from vision_transformer import *
from JigsawImageLoader import *


In [2]:
class classHead(nn.Module):
    def __init__(self, in_dim, num_classes):
        super().__init__()
        self.ln1=nn.LayerNorm(in_dim)
        self.lin1=nn.Linear(in_dim,num_classes)
        #self.sm=nn.Softmax(dim=1)
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)

    def forward(self, x,mask=None,ind_hidden=None):
        x=x[:,1,:]
        x=x.squeeze()
        x=self.ln1(x)
        x = self.lin1(x)
        #x = self.sm(x)
        return x

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=VisionTransformer()
model = model.to(device)
checkpoint=torch.load('checkpoint196.pth')
state_dict=checkpoint['model_sd']
red_state_dict={}
state_dict = {k.replace("backbone.", ""): v for k, v in state_dict.items()}
state_dict = {k: v for k, v in state_dict.items() if not k.startswith("head")}
model.load_state_dict(state_dict, strict=False)
for param in model.parameters():
    param.requires_grad=False
headClass=classHead(in_dim=768,num_classes=1000)
model= utils.MultiCropWrapper(model,headClass)
